In [10]:
from langchain.document_loaders import UnstructuredPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain_chroma import Chroma
import langchain

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from openai import OpenAI
import pandas as pd
import transformers
import openai
import torch

from importlib.metadata import version, PackageNotFoundError
from typing import Optional, List, Union, Dict, Any
from dotenv import load_dotenv
from datetime import datetime
from loguru import logger
from pathlib import Path
from enum import Enum
from tqdm import tqdm
import platform
import shutil
import os 

# langchain-huggingface 같은 플러그인 형태의 라이브러리를 알기 위한 함수입니다. 
def get_version(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return "N/A"

logger.debug(f"Python version        : {platform.python_version()}")
logger.debug(f"PyTorch version       : {torch.__version__}")
logger.debug(f"Transformers version  : {transformers.__version__}")
logger.debug(f"LangChain version     : {langchain.__version__}")
logger.debug(f"langchain-huggingface version : {get_version('langchain-huggingface')}")
logger.debug(f"langchain-openai version       : {get_version('langchain-openai')}")
logger.debug(f"langchain-chroma version       : {get_version('langchain-chroma')}")
logger.debug(f"OpenAI version        : {openai.__version__}")
logger.debug(f"Pandas version        : {pd.__version__}")

2025-05-31 19:20:59.776 | DEBUG    | __main__:<module>:36 - Python version        : 3.11.11
2025-05-31 19:20:59.778 | DEBUG    | __main__:<module>:37 - PyTorch version       : 2.7.0+cu126
2025-05-31 19:20:59.779 | DEBUG    | __main__:<module>:38 - Transformers version  : 4.52.3
2025-05-31 19:20:59.780 | DEBUG    | __main__:<module>:39 - LangChain version     : 0.3.25
2025-05-31 19:20:59.783 | DEBUG    | __main__:<module>:40 - langchain-huggingface version : 0.2.0
2025-05-31 19:20:59.785 | DEBUG    | __main__:<module>:41 - langchain-openai version       : 0.3.18
2025-05-31 19:20:59.787 | DEBUG    | __main__:<module>:42 - langchain-chroma version       : 0.2.4
2025-05-31 19:20:59.788 | DEBUG    | __main__:<module>:43 - OpenAI version        : 1.82.1
2025-05-31 19:20:59.789 | DEBUG    | __main__:<module>:44 - Pandas version        : 2.2.3


In [2]:
# .env의 내용을 모두 환경 변수로 접근 가능하게 만듦
load_dotenv()

# .env의 정보를 대문자 변수로 저장합니다.
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
HF_API_KEY=os.getenv("HF_API_KEY")

TRAIN_CSV=os.getenv("TRAIN_CSV")
TRAIN_SOURCE_DIR=os.getenv("TRAIN_SOURCE_DIR")
TEST_CSV=os.getenv("TEST_CSV")
TEST_SOURCE_DIR=os.getenv("TEST_SOURCE_DIR")
SAMPLE_CSV=os.getenv("SAMPLE_CSV")
SUBMISSION_PATH=os.getenv("SUBMISSION_PATH")

VECTORDB_BASE=os.getenv("VECTORDB_BASE")

CHUNK_SIZE=os.getenv("CHUNK_SIZE")
CHUNK_OVERLAP=os.getenv("CHUNK_OVERLAP")

# VALIDATION_MODE = true => openai 검증 모델 사용
# VALIDATION_MODE = false => huggingface 모델 사용
VALIDATION_MODE = os.getenv("VALIDATION_MODE").lower() == "true"

# SAMPLES = all => 마지막에 모든 샘플 추론
# SAMPLES = 특정 숫자 => 그 수까지만 추론
SAMPLES = os.getenv("SAMPLES", "all")
if SAMPLES.lower() == "all":
    sample_limit = None 
else:
    try:
        sample_limit = int(SAMPLES)
    except ValueError:
        raise ValueError(f"Invalid SAMPLES value: {SAMPLES}")   
    
HUGGINGFACE_EMBEDDING_MODEL=os.getenv("HUGGINGFACE_EMBEDDING_MODEL")
OPENAI_EMBEDDING_MODEL=os.getenv("OPENAI_EMBEDDING_MODEL")

HUGGINGFACE_LANGUAGE_MODEL=os.getenv("HUGGINGFACE_LANGUAGE_MODEL")
OPENAI_LANGUAGE_MODEL=os.getenv("OPENAI_LANGUAGE_MODEL")

In [3]:
# ENUM으로 텍스트 데이터를 관리합니다.
class PDFParseMethod(str, Enum):
    UNSTRUCTURED = "UnstructuredPdfLoader"
    PDFPLUMBER = "PdfPlumberLoader"
    
class EmbeddingModel(str, Enum):
    OPENAI = "openai"
    HUGGINGFACE = "huggingface"
    
class RetrievalMethod(str, Enum):
    SIMILARITY = "similarity"
    MMR = "mmr"
    BM25 = "bm25"
    ENSEMBLE = "ensemble"
    
class LanguageModel(str, Enum):
    OPENAI = "openai"
    HUGGINGFACE = "huggingface"

In [4]:
# 제공된 데이터를 저장하는 클래스
class DataProcessor:
    def __init__(
            self,
            train_csv=TRAIN_CSV,
            train_source_dir=TRAIN_SOURCE_DIR,
            test_csv=TEST_CSV,
            test_source_dir=TEST_SOURCE_DIR,
            sample_csv=SAMPLE_CSV
        ):
        self.train_df = pd.read_csv(train_csv)
        self.test_df = pd.read_csv(test_csv)
        self.submission_df = pd.read_csv(sample_csv)
        self.train_source_dir = train_source_dir
        self.test_source_dir = test_source_dir
    
    def get_train_df(self):
        return self.train_df
    def get_test_df(self):
        return self.test_df
    def get_submission_df(self):
        return self.submission_df
    def get_train_source_dir(self):
        return self.train_source_dir
    def get_test_source_dir(self):
        return self.test_source_dir
        
    @staticmethod
    def _get_pdf_path(directory: str) -> list:
        dir_path = Path(directory)
        if not dir_path.is_dir():
            logger.error(f"[_get_pdf_path] Invalid directory: {directory}")
            return []

        pdf_paths = [p for p in dir_path.rglob("*.pdf")]
        for path in pdf_paths:
            logger.debug(f"[_get_pdf_path] Found PDF: {path}")
        return pdf_paths
    
    def convert_all_pdfs(
            self,
            file_path: str, 
            chunk_size: int = int(CHUNK_SIZE),
            chunk_overlap: int = int(CHUNK_OVERLAP),
            method: PDFParseMethod = PDFParseMethod.PDFPLUMBER,
        ):
        
        pdf_path = DataProcessor._get_pdf_path(file_path)
        splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        
        if method == PDFParseMethod.UNSTRUCTURED:
            output = {}
            for path in pdf_path:
                key = path.name
                try:
                    loader = UnstructuredPDFLoader(str(path), mode="elements")
                    docs = loader.load()
                    chunks = splitter.split_documents(docs)
                except Exception as e:
                    logger.warning(f"[convert_all_pdfs: {method}] Failed to process {path}: {e}")
                    chunks = []
                finally:
                    output[key] = {
                        "chunks" : chunks,
                        "path" : str(path),
                    }
        
        elif method == PDFParseMethod.PDFPLUMBER:
            output = {}
            for path in pdf_path:
                key = path.name
                try:
                    loader = PDFPlumberLoader(str(path))  
                    docs = loader.load()
                    chunks = splitter.split_documents(docs)
                except Exception as e:
                    logger.warning(f"[convert_all_pdfs: {method}] Failed to process {path}: {e}")
                    chunks = []
                finally:
                    output[key] = {
                        "chunks": chunks,
                        "path": str(path),
                    }
 
        else:
            msg = f"Unsupported method: {method}"
            logger.error(msg)
            raise ValueError(msg)
        
        return output
        
data_processor = DataProcessor()
train_df = data_processor.get_train_df()
test_df = data_processor.get_test_df()
sample_df = data_processor.get_submission_df()

# TODO: 추후엔 pdf로 파싱한 데이터를 깔끔하게 전처리하는 로직을 한 번 돌려야 합니다. 
train_pdf = data_processor.convert_all_pdfs(data_processor.get_train_source_dir())
test_pdf = data_processor.convert_all_pdfs(data_processor.get_test_source_dir())

2025-05-31 19:11:45.513 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: data/train_source/재정통계해설.pdf
2025-05-31 19:11:45.514 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: data/train_source/국토교통부_소규모주택정비사업.pdf
2025-05-31 19:11:45.514 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: data/train_source/고용노동부_조기재취업수당.pdf
2025-05-31 19:11:45.515 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: data/train_source/국토교통부_민간임대(융자).pdf
2025-05-31 19:11:45.515 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: data/train_source/2024년도 성과계획서(총괄편).pdf
2025-05-31 19:11:45.516 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: data/train_source/보건복지부_노인일자리 및 사회활동지원.pdf
2025-05-31 19:11:45.517 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: data/train_source/고용노동부_청년일자리창출지원.pdf
2025-05-31 19:11:45.518 | DEBUG    | __main__:_get_pdf_path:37 - [_get_pdf_path] Found PDF: dat

In [5]:
# 크로마 기반 벡터스토어 저장
class EmbeddingVectorStoreBuilder:
    def __init__(
        self, 
        embedding_model: EmbeddingModel,
        huggingface_model_name: Optional[str] = HUGGINGFACE_EMBEDDING_MODEL,
        openai_model_name: Optional[str] = OPENAI_EMBEDDING_MODEL,
        huggingface_api_key: Optional[str] = HF_API_KEY,
        openai_api_key: Optional[str] = OPENAI_API_KEY,
        device: str = "cuda" if torch.cuda.is_available() else "cpu"
    ):
        self.embedding_model_type = embedding_model
        
        if embedding_model == EmbeddingModel.OPENAI:
            if not openai_api_key:
                msg = "OpenAI API key must be provided."
                logger.error(ValueError(msg))
                raise ValueError(msg)
            self.embedding = OpenAIEmbeddings(
                model=openai_model_name,
                openai_api_key=openai_api_key
            )
                
        elif embedding_model == EmbeddingModel.HUGGINGFACE:
            model_kwargs = {'device': device}
            if huggingface_api_key:
                model_kwargs["token"] = huggingface_api_key
            encode_kwargs = {'normalize_embeddings': True}
            
            self.embedding = HuggingFaceEmbeddings(
                model_name=huggingface_model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs    
            )
        else:
            msg = f"Unsupported embedding model: {embedding_model}"
            logger.error(ValueError(msg))
            raise ValueError(msg)
        
    def build(self, chunks: List[Document], persist_directory: Optional[Union[Path, str]] = None) -> Chroma:
        persist_directory = Path(persist_directory)
        persist_directory.mkdir(parents=True, exist_ok=True)
        logger.info(f"[build]: Building Chroma vector DB at {persist_directory}")
        vectordb = Chroma.from_documents(
            documents=chunks,
            embedding=self.embedding,
            persist_directory=str(persist_directory)
        )
        return vectordb

# OPENAI = "openai"
# HUGGINGFACE = "huggingface"
if VALIDATION_MODE:
    builder = EmbeddingVectorStoreBuilder(
        embedding_model=EmbeddingModel.OPENAI
    )
else:
    builder = EmbeddingVectorStoreBuilder(
        embedding_model=EmbeddingModel.HUGGINGFACE
    )

base = Path(VECTORDB_BASE) / builder.embedding_model_type
if base.exists():
    shutil.rmtree(base)

In [6]:
# pdf 별 벡터 db 저장
for key in train_pdf.keys():
    doc_dir = base / key
    _ = builder.build(
        chunks = train_pdf[key]['chunks'],
        persist_directory=doc_dir
    )
    
for key in test_pdf.keys():
    doc_dir = base / key
    _ = builder.build(
        chunks = test_pdf[key]['chunks'],
        persist_directory=doc_dir
    )

2025-05-31 19:13:17.792 | INFO     | __main__:build:43 - [build]: Building Chroma vector DB at chroma_db/EmbeddingModel.OPENAI/재정통계해설.pdf
2025-05-31 19:13:21.912 | INFO     | __main__:build:43 - [build]: Building Chroma vector DB at chroma_db/EmbeddingModel.OPENAI/국토교통부_소규모주택정비사업.pdf
2025-05-31 19:13:23.170 | INFO     | __main__:build:43 - [build]: Building Chroma vector DB at chroma_db/EmbeddingModel.OPENAI/고용노동부_조기재취업수당.pdf
2025-05-31 19:13:24.113 | INFO     | __main__:build:43 - [build]: Building Chroma vector DB at chroma_db/EmbeddingModel.OPENAI/국토교통부_민간임대(융자).pdf
2025-05-31 19:13:24.766 | INFO     | __main__:build:43 - [build]: Building Chroma vector DB at chroma_db/EmbeddingModel.OPENAI/2024년도 성과계획서(총괄편).pdf
2025-05-31 19:13:31.195 | INFO     | __main__:build:43 - [build]: Building Chroma vector DB at chroma_db/EmbeddingModel.OPENAI/보건복지부_노인일자리 및 사회활동지원.pdf
2025-05-31 19:13:31.987 | INFO     | __main__:build:43 - [build]: Building Chroma vector DB at chroma_db/EmbeddingModel.OPE

In [7]:
class RetrievalExecutor:
    def __init__(
        self,
        embedding_model: Union[OpenAIEmbeddings, HuggingFaceEmbeddings],
        base_directory: Union[str, Path]
    ):
        self.embedding_model = embedding_model
        self.base_directory = Path(base_directory)
        
    def _load_vectordb(self, db_key: str) -> Chroma:
        vectordb_path = self.base_directory / Path(db_key)
        if not vectordb_path.exists():
            msg = f"Vector DB not found ad {vectordb_path}"
            logger.error(msg)
            raise FileExistsError(msg)
        
        vectordb = Chroma(
            persist_directory=str(vectordb_path),
            embedding_function=self.embedding_model
        )
        
        return vectordb
    
    def _load_docs(self, vectordb: Chroma) -> List[Document]:
        data = vectordb.get(include=["documents", "metadatas"])
        return [
            Document(page_content=doc, metadata=meta)
                for doc, meta in zip(data["documents"], data["metadatas"])
        ]
    
    # TODO: 각종 인자를 .env로 관리해야 합니다. 
    def run(
        self,
        query: str,
        db_key: str,
        method: RetrievalMethod = RetrievalMethod.SIMILARITY,
        k: int = 5,
        alpha: float = 0.5
    ) -> List[Document]:
        
        vectordb = self._load_vectordb(db_key)
        
        if method == RetrievalMethod.SIMILARITY:
            return vectordb.similarity_search(query, k=k)
        
        elif method == RetrievalMethod.MMR:
            return vectordb.max_marginal_relevance_search(query, k=k)
        
        elif method == RetrievalMethod.BM25:
            docs = self._load_docs(vectordb)
            retriever = BM25Retriever.from_documents(docs)
            retriever.k = k
            return retriever.invoke(query)
        
        elif method == RetrievalMethod.ENSEMBLE:
            docs = self._load_docs(vectordb)
            bm25 = BM25Retriever.from_documents(docs)
            bm25.k = k
            dense = vectordb.as_retriever(search_kwargs={"k": k})
            ensemble = EnsembleRetriever(retrievers=[bm25, dense], weights=[1-alpha, alpha])
            return ensemble.invoke(query)
        
        else:
            msg = f"Unsupported method: {method}"
            logger.error(msg)
            raise ValueError(msg)
        
retriever = RetrievalExecutor(
    embedding_model=builder.embedding,
    base_directory=base
)

In [13]:
# TODO: 간헐적으로 병목이 걸리는데 그 원인을 아직 파악하지 못했습니다.
qa_pairs = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df), desc="[Retrieving] Document Retrieving"):
    question = row['Question']
    key = Path(row['Source_path']).name
    results = retriever.run(
        query=question,
        db_key=key,
        method=RetrievalMethod.ENSEMBLE,
    )
    context_parts = [f"### START-{i} ###{doc.page_content}### END-{i} ###" for i, doc in enumerate(results)]
    context = "".join(context_parts)
    qa_pairs.append({"context": context, "question": question})

100%|██████████| 98/98 [01:03<00:00,  1.55it/s]


In [15]:
class ChatResponder:
    def __init__(
        self,
        model_type: LanguageModel,
        huggingface_model_name: Optional[str] = HUGGINGFACE_LANGUAGE_MODEL,
        openai_model_name: Optional[str] = OPENAI_LANGUAGE_MODEL,
        huggingface_api_key: Optional[str] = HF_API_KEY,
        openai_api_key: Optional[str] = OPENAI_API_KEY,
        system_prompt: str = "당신은 정답을 친절하게 알려주는 비서입니다.",
        device: str = "cuda" if torch.cuda.is_available() else "cpu"        
    ):
        
        self.model_type = model_type
        self.system_prompt = system_prompt
        
        if model_type == LanguageModel.OPENAI:
            self.model = openai_model_name
            self.client = OpenAI(api_key=openai_api_key)
            
        elif model_type == LanguageModel.HUGGINGFACE:
            self.tokenizer = AutoTokenizer.from_pretrained(
                huggingface_model_name,
                token=huggingface_api_key
            )
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            self.model = AutoModelForCausalLM.from_pretrained(
                huggingface_model_name,
                token=huggingface_api_key,
                trust_remote_code=True
            ).to(device)

            
            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                device=0 if device == "cuda" else -1,
            )
        else:
            msg = f"Unsupported model type: {model_type}"
            logger.error(msg)
            raise ValueError(msg)
    
    # TODO: prompt_template과 system_prompt는 .txt로 가지고 있어야 합니다.
    # TODO: 각종 인자를 .env로 관리해야 합니다. 
    # TODO: batch_size 사용할 것인지 결정해야합니다.
    def run(
        self, 
        qa_pairs: List[Dict[str, str]], 
        prompt_template: str="아래 질문에 대하여 문맥에서 답을 찾아 {max_new_tokens}자 이내로 대답하세요. 질문: {question}, 문맥: {context}", 
        max_new_tokens: int=256, 
        do_sample: bool=True, 
        temperature: float=0.7, 
        batch_size: int=2,
        deliminator: str="##########"
    ) -> List[Dict[str, Any]]:
        output = []
        if self.model_type == LanguageModel.OPENAI:
            for pair in tqdm(qa_pairs, desc=f"[OpenAI Inference] model: {self.model}"):
                prompt = prompt_template.format(
                    question=pair['question'],
                    context=pair['context'],
                    max_new_tokens=max_new_tokens
                )
                prompt = prompt + deliminator
            
                response = self.client.responses.create(
                    model=self.model,
                    input=[
                        {"role": "developer", "content": self.system_prompt},
                        {"role": "user", "content": prompt}
                    ]
                )
                output_text = response.output_text.strip()
                input_tokens = response.usage.input_tokens
                output_tokens = response.usage.output_tokens
                output.append(
                    {
                        "input_text" : {
                            "system_prompt" : self.system_prompt,
                            "question" : pair['question'],
                            "context" : pair['context'],
                            "prompt_template" : prompt_template,
                        },
                        "output_text": output_text,
                        "input_tokens": input_tokens,
                        "output_tokens": output_tokens
                    }
                )            
            
        elif self.model_type == LanguageModel.HUGGINGFACE:
            prompts = [
                self.system_prompt + 
                "\n" + 
                prompt_template.format(
                    question=pair['question'],
                    context=pair['context'],
                    max_new_tokens=max_new_tokens
                ) + 
                deliminator
                    for pair in qa_pairs
            ]
            logger.info(f"[HuggingFace Inference Start] model: {self.model}")
            results = self.pipeline(
                prompts, 
                max_new_tokens=max_new_tokens, 
                do_sample=do_sample, 
                temperature=temperature,
                # batch_size=batch_size
            )
            logger.info(f"[HuggingFace Inference End] model: {self.model}")
            generated_texts = [r[0]["generated_text"] for r in results]
            output_texts = [text.split(deliminator)[-1].strip() for text in generated_texts]
            input_tokens_lst = [len(self.tokenizer.encode(prompt)) for prompt in prompts]
            output_tokens_lst = [len(self.tokenizer.encode(output)) for output in output_texts]
                    
            for pair, output_text, input_tokens, output_tokens in zip(qa_pairs, output_texts, input_tokens_lst, output_tokens_lst):
                output.append(
                    {
                        "input_text" : {
                            "system_prompt" : self.system_prompt,
                            "question" : pair['question'],
                            "context" : pair['context'],
                            "prompt_template" : prompt_template
                        },
                        "output_text": output_text,
                        "input_tokens": input_tokens,
                        "output_tokens": output_tokens,
                    }
                )  

        return output
    
if VALIDATION_MODE:
    responder = ChatResponder(
        model_type=LanguageModel.OPENAI,
    )
else:
    responder = ChatResponder(
        model_type=LanguageModel.HUGGINGFACE,
    )

outputs = responder.run(qa_pairs[:sample_limit])

[OpenAI Inference] model: gpt-4o-mini:   0%|          | 0/98 [00:00<?, ?it/s]

[OpenAI Inference] model: gpt-4o-mini: 100%|██████████| 98/98 [04:15<00:00,  2.61s/it]


In [16]:
sample_df.loc[range(len(outputs)), "Answer"] = [item['output_text'] for item in outputs]

timestamp = datetime.now().strftime("%Y%m%d%H%M")
submission_path = Path(SUBMISSION_PATH)
ext = submission_path.suffix
submission_path = Path(str(submission_path.with_suffix(""))+f"_{VALIDATION_MODE}_{timestamp}"+ext)
submission_dir = submission_path.parent
if not submission_dir.exists():
    submission_dir.mkdir(parents=True, exist_ok=True)
     
sample_df.to_csv(submission_path, index=False, encoding="utf-8-sig")